<a href="https://colab.research.google.com/github/JayAhn0104/Dash_Plolty/blob/master/Flower_DataCollect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from urllib.request import urlopen
import json
import pandas as pd
from tqdm import notebook
import easydict
import matplotlib.pyplot as plt

In [ ]:
def tue_fri_date_list(start, end):
  date_fri = pd.date_range(start, end, freq='W-FRI')
  date_tue = pd.date_range(start, end, freq='W-TUE')
  return pd.Series(date_fri.append(date_tue)).sort_values().astype(str)

def url_to_df(url, date):
  url_split = url.split('date')
  url_2 = url_split[0] + date + url_split[1]
  results = urlopen(url_2).read().decode("utf-8")
  df = pd.json_normalize(json.loads(results)['response']['items'])
  return df  

def df_concat(url, date_list):
  df_list = []
  for date in notebook.tqdm(date_list):
    new_df = url_to_df(url, date)
    if not new_df.empty:
      df_list.append(new_df)
    else:
      pass
  return pd.concat(df_list)

In [ ]:
your_api_key = '' # pleas enter your api key from https://flower.at.or.kr/
url = f"https://flower.at.or.kr/api/returnData.api?kind={your_api_key}&baseDate=date&flowerGubn=2&dataType=json&countPerPage=10000000"

In [ ]:
options = easydict.EasyDict({
    "url": url,
    "start" : '2017-01-01',
    "end" : '2021-12-31'
})

In [ ]:
date_list = tue_fri_date_list(options.start, options.end)
df = df_concat(options.url, date_list)

  0%|          | 0/522 [00:00<?, ?it/s]

In [ ]:
df[['maxAmt', 'minAmt', 'avgAmt', 'totAmt', 'totQty']] = df[['maxAmt', 'minAmt', 'avgAmt', 'totAmt', 'totQty']].astype(int)
df['saleYear'] = df['saleDate'].apply(lambda x: x.split('-')[0])
df['saleMonth'] = df['saleDate'].apply(lambda x: x.split('-')[1])
df['Year_Month'] = pd.to_datetime(df['saleYear'].astype('str') + '-' + df['saleMonth'].astype('str'))

In [ ]:
df.to_csv('2017_2021_flower.csv', encoding='euc-kr')